In [1]:
#Dice
#COMPAS for bail decision
#Adult-Income for income prediction
#German-Credit for assessing credit risk
#Dataset from Lending Club for loan decisions: https://www.lendingclub.com/info/download-data.action

#Alibi

#AIX360

## Importing libraries

In [2]:
import time
import pickle
import dice_ml
import scipy.io
import numpy as np
import pandas as pd
import seaborn as sns
import plotly.express as px
import matplotlib.pyplot as plt
import tensorflow.compat.v1 as tf

from sklearn.impute import *
from sklearn.metrics import *
from sklearn.ensemble import *
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import train_test_split
from sklearn.compose import make_column_selector as selector
from sklearn.preprocessing import OneHotEncoder, MaxAbsScaler, LabelEncoder

from imblearn.over_sampling import SMOTENC

from xmoai.problems.objectives import *
from xmoai.problems.restrictions import *
from xmoai.setup.configure import generate_counterfactuals_regression

from alibi.explainers import *

pd.set_option('display.max_columns', None)

In [3]:
tf.compat.v1.disable_eager_execution()
tf.keras.backend.clear_session()
tf.compat.v1.keras.backend.get_session().list_devices()

[_DeviceAttributes(/job:localhost/replica:0/task:0/device:CPU:0, CPU, 268435456, -1237164359454801192),
 _DeviceAttributes(/job:localhost/replica:0/task:0/device:GPU:0, GPU, 9396617216, -2876648868524212142)]

# Abalone Dataset

In [4]:
df_abalone = pd.read_csv('abalone.csv').dropna()
df_abalone

,Sex,Length,Diameter,Height,WholeWeight,ShuckedWeight,VisceraWeight,ShellWeight,Rings
0,M,0.455,0.365,0.095,0.5140,0.2245,0.1010,0.1500,15
1,M,0.350,0.265,0.090,0.2255,0.0995,0.0485,0.0700,7
2,F,0.530,0.420,0.135,0.6770,0.2565,0.1415,0.2100,9
3,M,0.440,0.365,0.125,0.5160,0.2155,0.1140,0.1550,10
4,I,0.330,0.255,0.080,0.2050,0.0895,0.0395,0.0550,7
...,...,...,...,...,...,...,...,...,...
4172,F,0.565,0.450,0.165,0.8870,0.3700,0.2390,0.2490,11
4173,M,0.590,0.440,0.135,0.9660,0.4390,0.2145,0.2605,10
4174,M,0.600,0.475,0.205,1.1760,0.5255,0.2875,0.3080,9
4175,F,0.625,0.485,0.150,1.0945,0.5310,0.2610,0.2960,10


## Defining required columns

In [5]:
num_indexes = []
cat_indexes = []

num_columns = df_abalone.drop(['Rings'], axis=1).columns.tolist()
cat_columns = ['Sex']
target = 'Rings'

for i in range(df_abalone.shape[1]):
    col = df_abalone.columns[i]
    if col in num_columns:
        num_indexes.append(i)
    elif col in cat_columns:
        cat_indexes.append(i)

### Converting string-encoded categories to integers

In [6]:
label_encoders = {}

for col in cat_columns:
    encoder = LabelEncoder().fit(df_abalone[col])
    label_encoders[col] = encoder
    df_abalone[col] = encoder.transform(df_abalone[col])

## Training

In [7]:
X_train, X_test, y_train, y_test = train_test_split(df_abalone.drop(target, axis=1), df_abalone[target], test_size=0.7, random_state=0)

In [8]:
one_hot_encode = False

# defining both numeric and categorical transformers
numeric_transformer = Pipeline(
    steps=[("imputer", KNNImputer()), ("scaler", MaxAbsScaler())]
)

# setting-up the preprocessing steps
if one_hot_encode:
    categorical_transformer = OneHotEncoder(handle_unknown='ignore', sparse=False)
    preprocessor = ColumnTransformer(
        transformers=[
            ("num", numeric_transformer, num_indexes),
            ("cat", categorical_transformer, cat_indexes),
        ]
    )
else:
    preprocessor = ColumnTransformer(
        transformers=[
            ("num", numeric_transformer, num_indexes + cat_indexes),
        ]
    )
    
# defining the model pipeline and training
model = Pipeline(
    steps=[("preprocessor", preprocessor),
           ("regressor", RandomForestRegressor(n_jobs=-1))]
).fit(X_train, y_train)

## Preparing counterfactual generation with DiCE

In [9]:
d = dice_ml.Data(dataframe=pd.concat([X_train, y_train], axis=1),
                 continuous_features=num_columns,
                 #continuous_features_precision=continuous_precision,
                 outcome_name=target)
m = dice_ml.Model(model=model, backend='sklearn', model_type='regressor')
exp = dice_ml.Dice(d, m)

## Preparing counterfactual generation with XMOAI

In [10]:
columns = X_train.columns
categorical_columns_one_hot_encoded = []
categorical_columns_label_encoded = {}

if one_hot_encode and type(model)==Pipeline:
    for cat_col in cat_columns:
        columns_in_cat = [col for col in columns if col.startswith(f'{cat_col}_')]
        columns_in_cat = np.argwhere(np.isin(columns, columns_in_cat)).flatten()
        
        if len(columns_in_cat) > 0:
            categorical_columns_one_hot_encoded.append(columns_in_cat)
else:
    for i in range(len(X_train.columns)):
        if X_train.columns[i] in cat_columns:
            categorical_columns_label_encoded[i] = np.sort(X_train[X_train.columns[i]].unique())
        
display(categorical_columns_one_hot_encoded)
display(categorical_columns_label_encoded)

[]

{0: array([0, 1, 2])}

In [11]:
# generating counterfactuals
immutable_column_indexes = [] # let's say we can't change the last column

upper_bounds = np.array(X_train.max(axis=0)*1.0) # this is the maximum allowed number per column
lower_bounds = np.array(X_train.min(axis=0)*1.0) # this is the minimum allowed number per column.
# you may change the bounds depending on the needs specific to the individual being trained.

## Generating counterfactuals

In [12]:
#y_test>y_train.median()
values = y_train.describe()
permitted_low = [values['min'], values['50%']]
permitted_high = [values['50%'], values['max']]

number_of_elements = 75
idx_low = np.argwhere(y_test.values<y_train.median()).flatten()[:number_of_elements]
idx_high = np.argwhere(y_test.values>y_train.median()).flatten()[:number_of_elements]

In [13]:
dice_all_exp_low = exp.generate_counterfactuals(X_test.iloc[idx_low], total_CFs=100, desired_range=permitted_high)
dice_all_exp_high = exp.generate_counterfactuals(X_test.iloc[idx_high], total_CFs=100, desired_range=permitted_low)

  4%|███▎                                                                               | 3/75 [01:12<28:45, 23.96s/it]

Only 98 (required 100)  Diverse Counterfactuals found for the given configuration, perhaps try with different parameters... ; total time taken: 00 min 23 sec


100%|██████████████████████████████████████████████████████████████████████████████████| 75/75 [29:53<00:00, 23.92s/it]


In [14]:
results = {}

In [ ]:
# individual to be evaluated
def calculate_metrics(df, X_current, y_desired, method, categorical_columns_label_encoded,
                      categorical_columns_one_hot_encoded):
    f1, prediction = get_difference_target_regression(model, df, y_desired, method)
    f2 = get_difference_attributes(df.values, X_current.values, ranges,
                                   categorical_columns_label_encoded,
                                   categorical_columns_one_hot_encoded)
    f3 = get_modified_attributes(df, X_current, categorical_columns_one_hot_encoded)
    
    return f1, f2, f3, prediction

ranges = (X_train.max() - X_train.min()).values

for index_to_refer in range(len(idx_low) + len(idx_high)):
    print(f'Processing {index_to_refer}')
    
    if index_to_refer < len(idx_low):
        X_current = X_test.iloc[[idx_low[index_to_refer]]]
        y_desired = values['75%']
        y_acceptable_range = permitted_high
    else:
        X_current = X_test.iloc[[idx_high[index_to_refer - len(idx_low)]]]
        y_desired = values['25%']
        y_acceptable_range = permitted_low

    # DiCE CFs
    print('Processing DiCE')
    try:
        if index_to_refer < len(idx_low):
            df_dice = dice_all_exp_low.cf_examples_list[index_to_refer].final_cfs_df.copy().drop(target, axis=1)
        else:
            df_dice = dice_all_exp_high.cf_examples_list[index_to_refer - len(idx_low)].final_cfs_df.copy().drop(target, axis=1)

        f1, f2, f3, prediction = calculate_metrics(df_dice, X_current, y_desired, 'predict',
                                                   categorical_columns_label_encoded,
                                                   categorical_columns_one_hot_encoded)

        for col in label_encoders.keys():
            df_dice[col] = label_encoders[col].inverse_transform(df_dice[col].astype(int))

        df_dice = pd.concat([df_dice, pd.DataFrame(np.vstack([f1, f2, f3]).T, columns=['F1', 'F2', 'F3'])], axis=1)
        df_dice['Algorithm'] = 'DiCE'
    except:
        df_dice = pd.DataFrame()
        display('Error in DiCE. Resuming...')

    # XMOAI CFs
    print('Processing XMOAI')
    try:
        front, X_generated, algorithms = generate_counterfactuals_regression(model, X_train,
                                  X_current.iloc[0], y_desired, immutable_column_indexes,
                                  y_acceptable_range, upper_bounds, lower_bounds,
                                  categorical_columns_label_encoded, categorical_columns_one_hot_encoded,
                                  num_indexes, n_gen=100, pop_size=100, max_changed_vars=5,
                                  verbose=False, select_best=True, seed=0)

        df_xmoai = pd.DataFrame(X_generated.copy(), columns=X_test.columns)
        for col in label_encoders.keys():
            df_xmoai[col] = label_encoders[col].inverse_transform(df_xmoai[col].astype(int))

        df_xmoai = pd.concat([df_xmoai, pd.DataFrame(front, columns=['F1', 'F2', 'F3'])], axis=1)
        df_xmoai['Algorithm'] = 'Proposal'
    except:
        df_xmoai = pd.DataFrame()
        display('Error in XMOAI. Resuming...')
    
    results[X_current.index[0]] = pd.concat([df_dice, df_xmoai]).reset_index(drop=True)

Processing 0
Processing DiCE
Processing XMOAI


invalid value encountered in power
invalid value encountered in power
invalid value encountered in power
invalid value encountered in power
invalid value encountered in power
invalid value encountered in power
invalid value encountered in power
invalid value encountered in power
invalid value encountered in power
invalid value encountered in power
invalid value encountered in power
invalid value encountered in power
invalid value encountered in power
invalid value encountered in power
invalid value encountered in power
invalid value encountered in power
invalid value encountered in power
invalid value encountered in power
invalid value encountered in power
invalid value encountered in power
invalid value encountered in power
invalid value encountered in power
invalid value encountered in power
invalid value encountered in power
invalid value encountered in power
invalid value encountered in power
invalid value encountered in power
invalid value encountered in power
invalid value encoun

Processing 1
Processing DiCE
Processing XMOAI


invalid value encountered in power
invalid value encountered in power
invalid value encountered in power
invalid value encountered in power
invalid value encountered in power
invalid value encountered in power
invalid value encountered in power
invalid value encountered in power
invalid value encountered in power
invalid value encountered in power
invalid value encountered in power
invalid value encountered in power
invalid value encountered in power
invalid value encountered in power
invalid value encountered in power
invalid value encountered in power
invalid value encountered in power
invalid value encountered in power
invalid value encountered in power
invalid value encountered in power
invalid value encountered in power
invalid value encountered in power
invalid value encountered in power
invalid value encountered in power
invalid value encountered in power
invalid value encountered in power
invalid value encountered in power
invalid value encountered in power
invalid value encoun

Processing 2
Processing DiCE
Processing XMOAI


...

invalid value encountered in power
invalid value encountered in power
invalid value encountered in power
invalid value encountered in power
invalid value encountered in power
invalid value encountered in power
invalid value encountered in power
invalid value encountered in power
invalid value encountered in power
invalid value encountered in power
invalid value encountered in power
invalid value encountered in power
invalid value encountered in power
invalid value encountered in power
invalid value encountered in power
invalid value encountered in power
invalid value encountered in power
invalid value encountered in power
invalid value encountered in power
invalid value encountered in power
invalid value encountered in power
invalid value encountered in power
invalid value encountered in power
invalid value encountered in power
invalid value encountered in power
invalid value encountered in power
invalid value encountered in power
invalid value encountered in power
invalid value encoun

Processing 7
Processing DiCE
Processing XMOAI


invalid value encountered in power
invalid value encountered in power
invalid value encountered in power
invalid value encountered in power
invalid value encountered in power
invalid value encountered in power
invalid value encountered in power
invalid value encountered in power
invalid value encountered in power
invalid value encountered in power
invalid value encountered in power
invalid value encountered in power
invalid value encountered in power
invalid value encountered in power
invalid value encountered in power
invalid value encountered in power
invalid value encountered in power
invalid value encountered in power
invalid value encountered in power
invalid value encountered in power
invalid value encountered in power
invalid value encountered in power
invalid value encountered in power
invalid value encountered in power
invalid value encountered in power
invalid value encountered in power
invalid value encountered in power
invalid value encountered in power
invalid value encoun

Processing 8
Processing DiCE
Processing XMOAI


invalid value encountered in power
invalid value encountered in power
invalid value encountered in power
invalid value encountered in power
invalid value encountered in power
invalid value encountered in power
invalid value encountered in power
invalid value encountered in power
invalid value encountered in power
invalid value encountered in power
invalid value encountered in power
invalid value encountered in power
invalid value encountered in power
invalid value encountered in power
invalid value encountered in power
invalid value encountered in power
invalid value encountered in power
invalid value encountered in power
invalid value encountered in power
invalid value encountered in power
invalid value encountered in power
invalid value encountered in power
invalid value encountered in power
invalid value encountered in power
invalid value encountered in power
invalid value encountered in power
invalid value encountered in power
invalid value encountered in power
invalid value encoun

In [ ]:
import pickle
pickle.dump([results, X_test.iloc[np.concatenate([idx_low, idx_high])]], open('abalone_results.pkl', 'wb'))